In [4]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [54]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [56]:

# parse time stamps
df = pd.read_csv("data.csv", parse_dates=['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
# Extract datetime features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month

In [58]:
# Features to be used for the model
features = ['sensor_encoded', 'hour', 'day_of_week', 'month']

X = df[features]

In [ ]:
# Scaling featurus
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [46]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'timestamp'
df = df.sort_values('timestamp').reset_index(drop=True)

# Verify sorting
df.head()

,sensor,bucket,timestamp,value,type,sensor_encoded
0,livingroom_PIR,1_4_7,2024-11-20 21:41:17.408,livingroombedarea,sensor,2
1,kitchen_PIR,1_5_9,2024-11-20 22:13:54.240,kitchen,sensor,1
2,kitchen_PIR,1_5_9,2024-11-21 00:32:07.153,kitchen,sensor,1
3,livingroom_PIR,1_4_7,2024-11-21 00:59:16.319,livingroombedarea,sensor,2
4,kitchen_PIR,1_5_9,2024-11-21 02:50:16.250,kitchen,sensor,1


In [47]:
# Create a flag that indicates when the 'value' changes compared to the previous row
df['room_change'] = (df['value'] != df['value'].shift(1)).astype(int)

# Create a cumulative sum of the 'room_change' flag to assign a unique group ID to each consecutive block
df['group_id'] = df['room_change'].cumsum()

In [53]:
# Group by 'group_id' and 'value' to handle each room separately
duration_df = df.groupby(['group_id', 'value']).agg(
    start_time=('timestamp', 'min'),
    end_time=('timestamp', 'max')
).reset_index()

# Calculate duration as the difference between end_time and start_time
duration_df['duration'] = duration_df['end_time'] - duration_df['start_time']


In [52]:
duration_df.head(50)

,group_id,value,start_time,end_time,duration
0,1,livingroombedarea,2024-11-20 21:41:17.408,2024-11-20 21:41:17.408,0 days 00:00:00
1,2,kitchen,2024-11-20 22:13:54.240,2024-11-21 00:32:07.153,0 days 02:18:12.913000
2,3,livingroombedarea,2024-11-21 00:59:16.319,2024-11-21 00:59:16.319,0 days 00:00:00
3,4,kitchen,2024-11-21 02:50:16.250,2024-11-21 07:01:32.415,0 days 04:11:16.165000
4,5,livingroombedarea,2024-11-21 09:03:39.394,2024-11-21 15:18:02.984,0 days 06:14:23.590000
5,6,kitchen,2024-11-21 17:08:48.155,2024-11-21 18:19:52.962,0 days 01:11:04.807000
6,7,livingroombedarea,2024-11-21 18:31:19.969,2024-11-21 18:34:47.829,0 days 00:03:27.860000
7,8,kitchen,2024-11-21 18:40:48.871,2024-11-21 18:52:34.137,0 days 00:11:45.266000
8,9,bathroom,2024-11-21 18:58:21.270,2024-11-21 18:58:21.270,0 days 00:00:00
9,10,livingroombedarea,2024-11-21 21:24:42.967,2024-11-21 21:36:20.694,0 days 00:11:37.727000
